# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [2]:
! git clone https://github.com/DEHMANIMOHAMMED/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 4.41 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 300

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [4]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

# Estimation des coefficients $$\theta_k$$ avec la méthode des moindres carrés

La méthode des moindres carrés est une approche simple et efficace pour estimer les coefficients $$\theta_k$$ dans un modèle linéaire. Cette méthode consiste à minimiser la somme des carrés des résidus entre les valeurs prédites et les cibles. Le modèle linéaire est défini comme suit :

$$
t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon
$$


In [5]:
import numpy as np
from utils_exercices import generate_dataset

# Charger les données
train_set, test_set = generate_dataset(mySeed)

# Préparer les données d'entraînement
X_train = train_set['inputs']  # Prédicteurs
t_train = train_set['targets']  # Cibles

# Ajouter une colonne de 1 pour le biais
X_train_aug = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

# Estimation des coefficients avec la formule des moindres carrés
Theta = np.linalg.inv(X_train_aug.T @ X_train_aug) @ X_train_aug.T @ t_train

# Afficher les coefficients
print("Coefficients estimés (Theta):", Theta)


Coefficients estimés (Theta): [15.01770145  2.97230274  2.9365536   5.85815285]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [9]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Couche d'entrée : 3 neurones (x, y, z)
        # Couche de sortie : 1 neurone pour prédire t
        # Pas de couches cachées car le modèle est linéaire
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        # Appliquer la transformation linéaire
        return self.linear(x)


**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [10]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        predictions = mySimpleNet(batch_inputs)

        # Compute the loss
        loss = criterion(predictions, batch_targets.unsqueeze(1))

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

    # Print loss every 50 epochs
    if epoch % 50 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [0/500], Loss: 226.9352
Epoch [50/500], Loss: 3.9363
Epoch [100/500], Loss: 3.0788
Epoch [150/500], Loss: 4.4658
Epoch [200/500], Loss: 3.7577
Epoch [250/500], Loss: 4.2155
Epoch [300/500], Loss: 3.9943
Epoch [350/500], Loss: 4.3130
Epoch [400/500], Loss: 4.4065
Epoch [450/500], Loss: 3.6162


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [11]:
# Extraire les poids et le biais
theta_1_3 = mySimpleNet.linear.weight.data.squeeze().numpy()  # Poids pour x, y, z
theta_0 = mySimpleNet.linear.bias.data.item()  # Biais (theta_0)

# Afficher les coefficients estimés
print("Estimations des coefficients (theta_k) :")
print(f"theta_0 (biais) : {theta_0:.4f}")
print(f"theta_1, theta_2, theta_3 (poids) : {theta_1_3}")


Estimations des coefficients (theta_k) :
theta_0 (biais) : 15.0198
theta_1, theta_2, theta_3 (poids) : [2.9730594 2.9340594 5.859893 ]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [12]:
from sklearn.metrics import mean_squared_error

# Données de test
X_test = test_set['inputs']
t_test = test_set['targets']

# Prédictions de la méthode des moindres carrés (Q1)
Theta_q1 = [15.01770145, 2.97230274, 2.9365536, 5.85815285]
predictions_q1 = Theta_q1[0] + X_test @ Theta_q1[1:]

# Prédictions du réseau neuronal (Q4)
predictions_q4 = mySimpleNet(torch.tensor(X_test, dtype=torch.float32)).detach().numpy().squeeze()

# Calcul des MSE
mse_q1 = mean_squared_error(t_test, predictions_q1)
mse_q4 = mean_squared_error(t_test, predictions_q4)

# Résultats
print(f"Erreur quadratique moyenne (MSE) - Moindres carrés (Q1) : {mse_q1:.4f}")
print(f"Erreur quadratique moyenne (MSE) - Réseau neuronal (Q4) : {mse_q4:.4f}")


Erreur quadratique moyenne (MSE) - Moindres carrés (Q1) : 4.0957
Erreur quadratique moyenne (MSE) - Réseau neuronal (Q4) : 4.0967


# Comparaison des performances entre les méthodes des moindres carrés (Q1) et le réseau neuronal (Q4)

## Résultats
- **Erreur quadratique moyenne (MSE) - Moindres carrés (Q1)** : 4.0957
- **Erreur quadratique moyenne (MSE) - Réseau neuronal (Q4)** : 4.0967

## Analyse
1. **Performances similaires :**
   - Les deux approches montrent des erreurs quadratiques moyennes très proches (moins de 0.001 de différence).
   - Cela indique que les deux méthodes capturent efficacement la relation entre les prédicteurs et les cibles.

2. **Léger avantage pour la méthode des moindres carrés :**
   - La méthode analytique donne une MSE légèrement inférieure, car elle calcule directement les coefficients optimaux en minimisant explicitement l'erreur quadratique.

3. **Impact du réseau neuronal :**
   - Le réseau neuronal, entraîné par SGD, approche bien la solution optimale mais peut légèrement diverger en raison de :
     - La stochasticité dans l'optimisation (SGD).
     - La convergence approximative par itérations.
     - Une sensibilité au taux d'apprentissage ou au nombre d'époques.

## Conclusion
- Pour ce problème simple de régression linéaire, la méthode des moindres carrés est légèrement plus précise, rapide et directe.
- Cependant, le réseau neuronal reste une solution flexible qui pourrait être adapté à des problèmes non linéaires ou plus complexes.

**Recommandation** : Utiliser les moindres carrés pour des problèmes simples et linéaires, et recourir à des réseaux neuronaux pour des cas plus complexes ou non linéaires.


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [15]:
# Nb de paramètres dans self.Down1: (calcul "à la main")
# Première convolution : 64 * 128 * 3 + 128
# Première BatchNorm : 128 * 2
# Deuxième convolution : 64 * 128 * 3 + 128
# Deuxième BatchNorm : 128 * 2
# Total : 49,920 paramètres

# Nb de paramètres au total:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
model = causalFCN()
total_params = count_parameters(model)
print(f"Nombre total de paramètres : {total_params}")

Nombre total de paramètres : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

# **Q2 : Réduction et restitution de la taille du vecteur d'entrée**

## Réduction de la taille (Encoder)
La taille du vecteur d'entrée est réduite par les mécanismes suivants dans la première partie du réseau (encoder) :
1. **Pooling causal (Down_causal)** :
   - Réduit la taille temporelle grâce à des couches de pooling (e.g., `MaxPool1d`).
   - Assure la causalité en ajustant le padding avant le pooling.

2. **Dilation** :
   - Les convolutions causales dilatées augmentent le champ réceptif sans réduire directement la taille, mais elles préparent l’information pour des résolutions plus basses.

## Restitution de la taille (Decoder)
La taille du vecteur d'entrée est restituée dans la deuxième partie du réseau (decoder) grâce aux mécanismes suivants :
1. **Upsampling (Up_causal)** :
   - Les couches `Up_causal` effectuent une interpolation ou une dé-convolution pour augmenter la résolution temporelle à chaque étape.

2. **Concaténation des features** :
   - À chaque étape, les features remontées sont concaténées avec celles du niveau correspondant dans l'encoder via des connexions skip, permettant une meilleure restitution des détails temporels.

## Conclusion
La réduction et la restitution de la taille sont soigneusement orchestrées pour préserver la causalité tout en capturant les dépendances à différentes échelles temporelles.


**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [17]:
import torch
import torch.nn.functional as F

# Initialisation du modèle causal
model = causalFCN()

# Taille de l'entrée
input_size = 10000

# Création de deux entrées identiques
input1 = torch.zeros(1, 1, input_size)  # Série temporelle nulle
input2 = input1.clone()

# Modifier uniquement la composante x[4500] de input2
input2[0, 0, 4500] = 1.0

# Passer les deux entrées dans le modèle
output1 = model(input1)
output2 = model(input2)

# Calcul de la différence entre les sorties
diff = (output1 - output2).abs()

# Identifier la composante non nulle associée à y[5000]
receptive_field_indices = torch.nonzero(diff[0, 0, 5000:], as_tuple=True)[0]
receptive_field_start = 5000 - torch.nonzero(diff[0, 0, :5000], as_tuple=True)[0].max()

# Champ réceptif
receptive_field = receptive_field_indices.max().item() + receptive_field_start + 1

print(f"Taille du champ réceptif associé à y[5000] : {receptive_field}")


Taille du champ réceptif associé à y[5000] : 5001


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?